# Session 2 - Temporal Network Analysis and Visualisation

**Ingo Scholtes**  
Chair of Machine Learning for Complex Networks  
Center for Artificial Intelligence and Data Science (CAIDAS)  
Julius-Maximilians-Universität Würzburg, DE  


**June 13 2022**

### Introducing the `TemporalNetwork` class

We have considered the `Paths` class, which is useful if you have direct access to path statistics in your time series data. This includes clickstreams of users in information networks, origin-destination statistics in transportation networks, flight ticket sequences, or other **collections of short, ordered sequences**.

In this unit, we expand this view towards temporal networks, i.e. high-resolution time series network data, where edges carry fine-grained time stamps. Considering technical, social, and biological systems that can be modelled as dynamic networks, such data cover a broad class of complex systems that can be studied with higher-order network models.

`pathpy` provides special support for the analysis of temporal networks data via its `TemporalNetwork` class. It is suitable for data that captures time-stamped edges $(v, w, t)$ instantaneously occurring at discrete time stamps $t$. Let us start by creating an empty instance of this class.

<span style="color:red">**TODO:** Import the package `pathpy` and rename it to `pp`. Create a new instance `t` of the class `TemporalNetwork` and print a summary of the instance.</span>

In [23]:
import pathpy as pp

t = pp.TemporalNetwork()
print(t)

Nodes:			0
Time-stamped links:	0
Links/Nodes:		N/A



The `TemporalNetwork` instance `t` stores two key information: a list of nodes `t.nodes` and a collection `t.tedges` of time-stamped edges  $(v, w, t)$. Let us add some time-stamped edges to this instance.

<span style="color:red">**TODO:** Use the `add_edge` function to add six (directed) time-stamped edges $(a,b, 1), (b, a, 3), (b, c, 3), (d, c, 4), (c, d, 5), (c, b, 6)$ and print the result.</span>

In [25]:
t.add_edge('a', 'b', 1)
t.add_edge('b', 'a', 3)
t.add_edge('b', 'c', 3)
t.add_edge('d', 'c', 4)
t.add_edge('c', 'd', 5)
t.add_edge('c', 'b', 6)
print(t)

Nodes:			4
Time-stamped links:	12
Links/Nodes:		3.0
Observation period:	[1, 6]
Observation length:	 5 
Time stamps:		 5 
Avg. inter-event dt:	 1.25
Min/Max inter-event dt:	 1/2


We get basic summary statistics, such as the number of time-stamped interactions, the minimum and maximum timestamp, the duration of the observation, the number of different timestamps, as well as the average, minimum, and maximum time difference between consecutive time-stamped edges.

Above we used integer timestamps, which represent **discrete time units**. But we often have data where time is given in terms of a date and/or time of day. Luckily, `pathpy` supports arbitrary timestamp formats. Let us try this in an example.

<span style="color:red">**TODO:** Create a new `TemporalNetwork` instance `t_realtime` and add three time-stamped edges with string timestamps in the format "YYYY-MM-DD HH:mm:SS". Print the resulting instance and print all time-stamped edges.</span>

In [26]:
t_realtime = pp.TemporalNetwork()
t_realtime.add_edge('a', 'b', '2018-08-22 09:30:22')
t_realtime.add_edge('b', 'c', '2018-08-22 09:30:25')
t_realtime.add_edge('c', 'a', '2018-08-22 10:30:25')
print(t_realtime)

for e in t_realtime.tedges:
    print(e)

Nodes:			3
Time-stamped links:	3
Links/Nodes:		1.0
Observation period:	[1534923022, 1534926625]
Observation length:	 3603 
Time stamps:		 3 
Avg. inter-event dt:	 1801.5
Min/Max inter-event dt:	 3/3600
('a', 'b', 1534923022)
('b', 'c', 1534923025)
('c', 'a', 1534926625)


We observe that `pathpy` internally converts such timestamps into UNIX timestamps. For custom formats, we can set a custom `timestamp_format` parameter that will be used for this conversion. After the conversion, all time units will be in seconds (see e.g. the min/max inter-event time).

Just like other `pathpy` objects, we can directly embed interactive visualisations of a `TemporalNetwork` in-line in `jupyter`. Let us try this with our first example instance `t`.

<span style="color:red">**TODO:** Visualise the `TemporalNetwork` instance `t` by writing the instance variable in an empty `jupyter` cell.</span>

In [27]:
t

Using the default parameters, this visualisation is too fast. Luckily, we can use the generic `pp.visualisation.plot` function to pass a `style` for the visualisation. We can use all parameters that we used for static networks, plus additional parameters that **influence temporal aspects of the visualisation**. 

Of particular importance are the parameters `ms_per_frame` and `ts_per_frame`: The first specifies how many time units will be shown in one frame of the visualisation, allowing us to compress the visualisation by showing multiple timestamps in a single frame. This is helpful when you want to coarse-grain visualisations of high-resolution temporal network data. The parameter `ms_per_frame` defines the target frame rate of the visualisation by adjusting how many milliseconds each frame is displayed. 

Two more parameters will influence the force-directed layout algorithm, that is used to position nodes in the network. In a temporal network, the question is which time-stamped edges should be taken into account for the force-calculation at any given time stamp. If we only consider currently active edges, the layout will change too fast to recognize interesting patterns. If we consider all edges at every time step, node positions will be static despite the dynamics of edges. In real settings we want a compromise between those extremes, i.e. we specify a time window around the current time stamp within which edges are taken into account in the force-directed layout calculation. We can achieve this by setting the number of timestamps to consider before and after the currently shown frame via the `look_ahead` and `look_behind` parameters. 

Finally, we can style active and inactive nodes and edges individually via the parameters `active_edge_width`, `inactive_edge_width`, `active_node_color`, and `inactive_node_color`. This allows us to change the color and/or size of nodes/edges whenever they are involved in an interaction.

<span style="color:red">**TODO:** Create a visualisation where a single timestamp is shown per frame, and each frame is shown for 2 seconds. For the force-directed layout, consider edges active up to two time units before and after the current timestamp. Increase the thickness of currently active egdes. </span>

In [28]:
style = {    
    'ts_per_frame': 1, 
    'ms_per_frame': 2000,
    'look_ahead': 2, 
    'look_behind': 2, 
    'node_size': 15, 
    'inactive_edge_width': 2,
    'active_edge_width': 4, 
    'label_color' : '#ffffff',
    'label_size' : '24px',
    'label_offset': [0,5]
    }
pp.visualisation.plot(t, **style)

Again, this generates an embedded **interactive** visualisation, i.e. you can pan and zoom, or drag nodes. The controls in the top part of the visualisation allow you to stop, start or restart the simulation. 

We can easily save such interactive visualisations as stand-alone HTML5 files, which can be distributed via the Web.

<span style="color:red">**TODO:** Save the visualisation from above to a file and open it in a browser. </span>

In [29]:
pp.visualisation.export_html(t, '../visualisations/2_temporal_network.html', **style)

In [30]:
pp.visualisation.export_tikz(t, '../visualisations/2_temporal_network.tex')

### Calculating path statistics in temporal networks

As explained in the lecture, the ordering and timing in which time-stamped edges occur in a `TemporalNetwork` gives rise to so-called **causal or time-respecting paths**. In a nutshell, for two time-stamped edges $(a, b, t)$ and $(b, c, t')$ to contribute to a causal path $a \rightarrow b \rightarrow c$ it must hold that $t < t'$. If we swap timestamps such that the edge $(b, c)$ occurs **before** (a,b), no causal path $a \rightarrow b \rightarrow c$ exists.

The chronological order of time-stamped edges crucially influences causal paths, i.e. which nodes can possibly influence each other in time-stamped edge sequences. Moreover, we often want to limit the **maximum time difference between consecutive edges** that contribute to a causal path. For data on dynamic social interactions that spans several years, it does not make sense to consider all chronologically ordered edges as possible causal paths for the propagation of information. After all, humans have limited memory and we should thus consider interactions that occur far apart in time as independent.

We can formally add this condition by setting a maximum time difference $\delta$ for the path calculation. That is, we only consider two edges $(a, b, t)$ and $(b, c, t')$ to contribute to a causal path $a \rightarrow b \rightarrow c$ if $ 0 < t' - t \leq \delta$.

With this definition at hand, and by setting our maximum time difference $\delta$, we can now **calculate causal path statistics in time-stamped network data**. `pathpy` provides powerful algorithms to calculate (or estimate) causal path statistics based on a `TemporalNetwork` instance. Let us try this in our toy example.

 <span style="color:red">**TODO:** Use the method `pp.path_extraction.paths_from_temporal_network_dag` to calculate causal path statistics for the example temporal network `t` and a maximum time difference $\delta=1$. Print the resulting `Paths` object. as well as all causal paths.</span>

In [31]:
causal_paths = pp.path_extraction.temporal_paths.paths_from_temporal_network_dag(t, delta=1)
print(causal_paths)

for l in causal_paths.paths:
    for p in causal_paths.paths[l]:
        if causal_paths.paths[l][p][1]>0:
            print('{0} -> {1}'.format(p, causal_paths.paths[l][p][1]))

Directed Acyclic Graph
Nodes:		10
Roots:		4
Leaves:		5
Links:		6
Acyclic:	None

Total path count: 		5.0 
[Unique / Sub paths / Total]: 	[5.0 / 13.0 / 18.0]
Nodes:				4 
Edges:				6
Max. path length:		2
Avg path length:		1.2 
Paths of length k = 0		0.0 [ 0.0 / 11.0 / 11.0 ]
Paths of length k = 1		4.0 [ 4.0 / 2.0 / 6.0 ]
Paths of length k = 2		1.0 [ 1.0 / 0.0 / 1.0 ]

('d', 'c', 'd') -> 1.0
('a', 'b') -> 1.0
('c', 'b') -> 1.0
('b', 'c') -> 1.0
('b', 'a') -> 1.0


For $\delta=1$, it is easy to verify that this output is correct. After all, there is only one pair of (directed) edges $(d, c, 4), (c, d, 5)$ that contributes to a causal path of length two. In addition, we have four time-stamped edges, each of which is a trivial causal path of length one.

This brings us to an important observation: In line with what we have discussed in the previous session, time-aggregated models of temporal networks discard the ordering and timing of links. They are thus **maximum entropy, first-order network models for causal paths** in temporal networks.

While it is easy to understand the path statistics for a maximum time difference of $\delta=1$, already for $\delta=2$ the situation gets more complicated.

<span style="color:red">**TODO:** Generate and print all causal paths emerging for a maximum time difference $\delta=2$.</span>

In [33]:
causal_paths = pp.path_extraction.paths_from_temporal_network_dag(t, delta=2)
print(causal_paths)

for l in causal_paths.paths:
    for p in causal_paths.paths[l]:
        if causal_paths.paths[l][p][1]>0:
            print('{0} -> {1}'.format(p, causal_paths.paths[l][p][1]))

Directed Acyclic Graph
Nodes:		13
Roots:		2
Leaves:		8
Links:		12
Acyclic:	None

Total path count: 		4.0 
[Unique / Sub paths / Total]: 	[4.0 / 24.0 / 28.0]
Nodes:				4 
Edges:				6
Max. path length:		3
Avg path length:		2.25 
Paths of length k = 0		0.0 [ 0.0 / 13.0 / 13.0 ]
Paths of length k = 1		0.0 [ 0.0 / 9.0 / 9.0 ]
Paths of length k = 2		3.0 [ 3.0 / 2.0 / 5.0 ]
Paths of length k = 3		1.0 [ 1.0 / 0.0 / 1.0 ]

('d', 'c', 'b') -> 1.0
('d', 'c', 'd') -> 1.0
('a', 'b', 'a') -> 1.0
('a', 'b', 'c', 'd') -> 1.0


We now observe one causal path $a \rightarrow b \rightarrow c \rightarrow d$ of length three, and three additional causal paths of length two. All shorter causal paths are contained in those longer causal paths, as shown by the path statistics shown above.

While I will not go into the full details of `pathpy`'s path calculation algorithm, we can at least peek into how it is done. Internally, `pathpy` generates a so-called **time-unfolded** directed and acyclic graph (see definition and illustration in [this work](https://arxiv.org/abs/1208.0588)). This graph is then the basis to calculate causal path statistics. We can get an idea how this works by manually generating a time-unfolded graph from a temporal network.

<span style="color:red">**TODO:** Use the `pp.DAG.from_temporal_network` method to create a time-unfolded graph from the `TemporalNetwork` `t` for $\delta=2$. Generate a visualisation where you color all time-unfolded nodes according to the "real" node to which they belong and increase the size of root nodes so you can easily tell them apart.</span>

In [35]:
dag, node_map = pp.DAG.from_temporal_network(t, delta=2)

color_map = {'a': 'red',
             'b': 'green',
             'c': 'blue',
             'd': 'red'}

style = { 
    'width': 400, 
    'height': 300, 
    'node_color': { v: color_map[node_map[v]] for v in dag.nodes }, 
    'node_size': { v: 8 if v in dag.roots else 5 for v in dag.nodes }
}

pp.visualisation.plot(dag, **style)

We obtain a time-unfolded graph, where each node is a **temporal copy** of an actual node, and where colors indicate to which node each temporal copy belongs. Importantly, the resulting directed acyclic graph has two root nodes (shown as larger nodes) with degree zero. Each of these root nodes shows where a causal path can possibly start. These roots are the basis for `pathpy`'s path calculation algorithm, i.e. in the simplest case `pathpy` will simply process all of these results to generate the path statistics.

### Path analysis in real dynamic social networks

To simplify the analysis of large collections of time-stamped network data, `pathpy` natively supports `SQLite` databases. For this, let us import `python`'s `sqlite3` module and use the `connect` function to connect to a `SQLite` database file. In order to access columns by name rather than index, we also need to set the default `row_factory` on the `Connection` object that is returned by the `connect` function to `sqlite3.Row`.

<span style="color:red">**TODO:** Import the module `sqlite3` and connect to the `SQLite` database file `temporal_networks.db`, which we provide for you in the `/data` directory. Set the `row_factory` of the resulting connection to `row_factory`.</span>

In [36]:
import sqlite3
con = sqlite3.connect('../data/temporal_networks.db')
con.row_factory = sqlite3.Row

We can now generate temporal networks from an `SQLite` database. For this, we must pass a so-called `SQLite` cursor to the constructor of `TemporalNetwork`. This cursor essentially tells which rows and columns of the database should be used for the temporal network creation. We can create it by passing an `SQL` query to the `execute` function of the `connection` object.

 The `TemporalNetwork` class assumes that the source, target, and timestamp of time-stamped edges are stored in columns called `source`, `target` and `time`. If your database has a different format, you can use the `SQL` `as` keyword to rename the columns in the query. Note that you can use the extension [SQLite](https://marketplace.visualstudio.com/items?itemName=alexcvzz.vscode-sqlite) to browse the `SQLite database` file within Visual Studio Code. Once you have installed it,  hit `CTRL + Shift + P` and enter the command `SQLite: Open Database in Explorer`. This will bring up a new database panel in the explorer section on the left, which you can use to show the structure and contents of the database.
 
Let us now try this with a data set on (undirected) time-stamped proximity contacts between patients and workers in a hospital, collected via the [Sociopatterns](http://www.sociopatterns.com) project.
 
<span style="color:red">**TODO:** Create an undirected `TemporalNetwork` instance `t_ho` from an `SQL` query of the database `sociopatterns_hospital` and print the resulting object.</span>

In [37]:
t_ho = pp.TemporalNetwork.from_sqlite(con.execute('SELECT source, target, time FROM sociopatterns_hospital'),
                                      directed=False)
print(t_ho)

Nodes:			75
Time-stamped links:	64848
Links/Nodes:		864.64
Observation period:	[140, 347640]
Observation length:	 347500 
Time stamps:		 9453 
Avg. inter-event dt:	 36.76470588235294
Min/Max inter-event dt:	 20/26980


We see that this temporal network has more than 64,000 time-stamped contacts between 75 individuals. On average, each individual engages in more than 860 contacts and the average time difference between contacts is approx. 36 seconds. More interestingly, the minimum inter-event time is 20 seconds, which is due to the fact that the sensor equipment used registered contacts every 20 seconds.

Blindly calculating causal paths for large values of $\delta$ (say $\delta=300$ for a maximum time difference between contacts of five minutes) would lead to the generation of a huge time-unfolded graph. However, since the sampling interval in this data set is larger than the time unit of 1 second, we can use `pathpy`'s time rescaling feature. It helps us to more efficiently calculate causal paths by rescaling internal time units to match a data set's sampling frequency. In the Sociopatterns data, we can rescale time by a factor of 20 without any loss of information. This means that timestamps 20, 40, 80 will be mapped to integer time units 1, 2, 4. Let us try this:

<span style="color:red">**TODO:** Repeat the previous step, but rescale internal time units by a factor of 20. Print the resulting instance `t_ho`</span>

In [38]:
t_ho = pp.TemporalNetwork.from_sqlite(con.execute('SELECT source, target, time FROM sociopatterns_hospital'),
                                      directed=False, time_rescale=20)
print(t_ho)

Nodes:			75
Time-stamped links:	64848
Links/Nodes:		864.64
Observation period:	[7, 17382]
Observation length:	 17375 
Time stamps:		 9453 
Avg. inter-event dt:	 1.838235294117647
Min/Max inter-event dt:	 1/1349


The fact that the minimum inter-event time is now 1 shows that the internal time units have been scaled accordingly. Note that as long as we match the time rescaling to the ssampling frequency, this does not change path structure of the resulting temporal network. We must, however, account for the time unit when we specifiy the $\delta$ for the causal path calculation.

<span style="color:red">**TODO:** Calculate causal paths in the temporal network `t_ho` and set the maximum time difference to one minute.</span>

In [39]:
causal_paths = pp.path_extraction.paths_from_temporal_network_dag(t_ho, delta=3)
print(causal_paths)

Directed Acyclic Graph
Nodes:		85215
Roots:		8276
Leaves:		34570
Links:		194544
Acyclic:	None



KeyboardInterrupt: 

For this calculation, `pathpy` needs to analyse a time-unfolded graph with more than 85,000 nodes and more than 8276 root nodes, which are processed in the path calculation. This takes a moment, so let us skip it here by interrupting the kernel. In fact, depending on the size and temporal characteristics of your data set, exhaustively calculating all causal paths for large values of $\delta$ can quickly become prohibitive.

Instead of exhaustively calculating **all** causal paths we actually want an efficient method to get a quick **estimate** of causal path statistics, which we can then use to infer higher-order models for causal paths. `pathpy` provides a smart way to do this by randomly sampling roots in the time-unfolded acyclic graph. Let us try this for 50 root nodes.

<span style="color:red">**TODO:** Use the method `sample_paths_from_temporal_network_dag` to estimate causal path statistics in the temporal network `t_ho` for a maximum time difference of one minute, randomly sampling 15 roots in the time-unfolded graph.</span>

In [42]:
causal_paths = pp.path_extraction.sample_paths_from_temporal_network_dag(t_ho, delta=3, num_roots=15)
print(causal_paths)

Directed Acyclic Graph
Nodes:		85215
Roots:		8276
Leaves:		34570
Links:		194544
Acyclic:	None

Total path count: 		1405.0 
[Unique / Sub paths / Total]: 	[1405.0 / 2494442.0 / 2495847.0]
Nodes:				51 
Edges:				243
Max. path length:		129
Avg path length:		49.42846975088968 
Paths of length k = 0		0.0 [ 0.0 / 70852.0 / 70852.0 ]
Paths of length k = 1		4.0 [ 4.0 / 69443.0 / 69447.0 ]
Paths of length k = 2		5.0 [ 5.0 / 68037.0 / 68042.0 ]
Paths of length k = 3		10.0 [ 10.0 / 66631.0 / 66641.0 ]
Paths of length k = 4		10.0 [ 10.0 / 65235.0 / 65245.0 ]
Paths of length k = 5		12.0 [ 12.0 / 63847.0 / 63859.0 ]
Paths of length k = 6		11.0 [ 11.0 / 62472.0 / 62483.0 ]
Paths of length k = 7		12.0 [ 12.0 / 61107.0 / 61119.0 ]
Paths of length k = 8		11.0 [ 11.0 / 59755.0 / 59766.0 ]
Paths of length k = 9		13.0 [ 13.0 / 58412.0 / 58425.0 ]
Paths of length k = 10		15.0 [ 15.0 / 57080.0 / 57095.0 ]
Paths of length k = 11		16.0 [ 16.0 / 55762.0 / 55778.0 ]
Paths of length k = 12		15.0 [ 15.0 / 54461.0

While this should finish within a few seconds, we still get a selection of several thousand paths with varying path length, that we can use to generate higher-order models. We will use this approach, when we study real data in a later session.

### Bonus: Data-driven story-telling with custom visualisation templates

Finally, we briefly introduce custom visualisation templates, which you may find useful for data-driven and visual story-telling tasks. As an example, we will use a data set on character co-occurrences in the text of The Lord of the Rings. You can load it from the table `lotr` SQLite database. In this table, each row `source, target, time` captures that the characters `source` and `target` are mentioned within the same sentence, where `time` chronologically numbers sentences throughout all three novels.

In the following, we want to generate a nice interactive visualisation of this data set. For this, we will use the custom templating mechanism of `pathpy`. It allows you to define your own HTML templates, that you can derive from the default visualisation templates that we have used so far. This enables us to use the default `pathpy` visuals as a baseline, that we can tune to our needs. 

Technically, such a template is nothing more than an HTML5 file with embedded JavaScript and CSS code. `pathpy` will use this template, and replace placeholder variables that we can set via the `style` parameter dictionary. We can tell `pathpy` to use an arbitrary custom template file by setting the entry `style['template'] = filename`. In this template, we can then use variables in the form `$variable`, which we can set from within `python` by setting `style['variable'] = value`.

In the custom template file `data/custom_template.html` we use all of `pathpy`'s default style parameters, as well as two additional parameters `chapter_data` and `character_classes`. We will use the first to pass chapter marks to the visualisation, which are then shown in the top left part of the visualisation as the story unfolds. Moreover, we will visualise the different factions (Hobits, Elves, Fellowship, Dwarves, ...) to which characters belong, so we need to pass those to the template as well.

You can read the character and chapter data from the corresponding json-files in the data directory. Just use the `json.load` function in `python's` json file to read them into two dictionaries and pass those two dictionaries to the corresponding `style` parameters.

<span style="color:red">**TODO:** Use the `pp.visualisations.export_html()` method to create a visualisation of dynamic character interactions in The Lord of The Rings based on the table `lotr` in the SQLite database and the custom template file `custom_template.html` in the `data` folder.</span>

In [43]:
import json

t_lotr = pp.TemporalNetwork.from_sqlite(
    con.execute('SELECT source, target, time FROM lotr'))
print(t)

# Load chapter marks from JSON file
with open('../data/lotr_chapters.json', 'r') as f:
    chapters = json.load(f)

# Load character classes from JSON file
with open('../data/lotr_characters.json', 'r') as f:
    characters = json.load(f)

style = {
    # some default parameters
    'width': 1200,
    'height': 1000,
    'look_ahead': 500,
    'look_behind': 1500,
    'ts_per_frame': 20, 
    'ms_per_frame': 50,
    'inactive_edge_width': 4.0,
    'active_edge_width': 6.0,
    'label_offset': [0,-16],    
    'node_size': 10,
    'label_size': '14px',
    
     # tell pathpy to use a user-provided custom template
    'template': '../data/custom_template.html',
    
    # add custom parameters defined in our custom template
    'chapter_data': chapters, 
    'character_classes': characters,    
}

# generate HTML based on our custom template
pp.visualisation.export_html(t_lotr, filename='../visualisations/2_demo_lotr.html', **style)

Nodes:			4
Time-stamped links:	12
Links/Nodes:		3.0
Observation period:	[1, 6]
Observation length:	 5 
Time stamps:		 5 
Avg. inter-event dt:	 1.25
Min/Max inter-event dt:	 1/2


As a little distraction at the end of this session, open the generated file in your browser, lean back and enjoy watching the story unfold - as a temporal network :-)